# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Содержание исследования**

1. [Загрузка и подготовка данных](#1)
2. [Предобработка данных](#2)
    * [Поиск дубликатов](#2_1)
    * [Удаление лишней информации для обучения](#2_2)
    * [Аномальные значения в целевом признаке](#2_3)
3. [Обучение и проверка модели](#3)
4. [Подготовка к расчету прибыли](#4)
5. [Расчет прибыли и рисков](#5)
6. [Общий вывод](#6)

## Загрузка и подготовка данных <a id="1"></a>

In [1]:
# загрузка модулей для работы с датасетом
import pandas as pd
import numpy as np

In [2]:
# загрузка модулей для работы с МашОбом
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты и описание на месте
    
  
</div>


Познакомимся с нашими данными:

In [3]:
region_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
region_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
region_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [4]:
def take_info(df: pd.DataFrame, info: str):
    if info == 'info':
        display(df.head())
        df.info()
        display(df.describe())
    elif info == 'dup':
        print(f'Количество дубликатов: {df.duplicated().sum()}')

**Первый регион**

In [5]:
take_info(region_0, 'info')

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


**Второй регион**

In [6]:
take_info(region_1, 'info')

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


**Третий регион**

In [7]:
take_info(region_2, 'info')

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Информация о данных:

* `id` - уникальный идентификатор скважины;
* `f0`, `f1`, `f2` - три признака точек;
* `product` - объем запасов в скважине (тыс. баррелей)

**Вывод**

В процессе знакомства с данными определили следующее:
* для обучения модели имеются лишние признаки (столбец `id`). Необходимо его удалить для всех регионов;
* по целевому признаку лидером по объему скважин является третий регион. Более того, медианное значение целевого признака данного региона больше среди вссх остальных регионов, что говорит нам о том, что предположительно, этот регион имеет больше всего запасов в скважинах.
* имеется нулевое значение целевого признака во всех регионах - необходимо разобраться, почему у нас есть нулевые значения и выбрать наиболее подходящий способ для устранения проблемы.

## Предобработка данных <a id="2"></a>

### Поиск дубликатов<a id="2_1"></a>

Для начала проверим все датасеты на наличие дубликатов 

**Первый регион**

In [8]:
take_info(region_0, 'dup')

Количество дубликатов: 0


**Второй регион**

In [9]:
take_info(region_1, 'dup')

Количество дубликатов: 0


**Третий регион**

In [10]:
take_info(region_2, 'dup')

Количество дубликатов: 0


### Удаление лишней информации для обучения <a id="2_2"></a>

Так как явных дубликатов в датасетах нет, то удалим признак `id`:

In [11]:
region_0 = region_0.drop('id', 1)
region_1 = region_1.drop('id', 1)
region_2 = region_2.drop('id', 1)

Убедимся в том, что этот столбец удален:

In [12]:
list_to_col = [region_0, region_1, region_2]
list_str = ['Первый регион', 'Второй регион', 'Третий регион']
for i in range(len(list_to_col)):
    print(f'{list_str[i]}')
    print(f'{list_to_col[i].columns}')

Первый регион
Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Второй регион
Index(['f0', 'f1', 'f2', 'product'], dtype='object')
Третий регион
Index(['f0', 'f1', 'f2', 'product'], dtype='object')


Столбец во всех столбцах удален, рассмотрим на нулевые знанчения в целевом признаке.

### Аномальные значения в целевом признаке<a id="2_3"></a>

Рассмотрим все регионы.

**Первый регион**:

In [13]:
region_0[region_0['product'] == 0.0]

,f0,f1,f2,product
57263,-0.702064,0.375992,0.236572,0.0


**Второй регион**:

In [14]:
region_1[region_1['product'] == 0.0]

,f0,f1,f2,product
11,16.320755,-0.562946,-0.001783,0.0
13,6.695604,-0.749449,-0.007630,0.0
62,21.418478,-5.134490,-0.002836,0.0
63,6.822701,3.104979,-0.000723,0.0
77,6.750150,-11.893512,-0.001601,0.0
...,...,...,...,...
99936,5.085749,-3.980305,0.005063,0.0
99948,8.277805,-9.178818,0.003275,0.0
99956,13.343983,-1.290200,0.005980,0.0
99961,13.854163,-11.528089,-0.005556,0.0


**Третий регион**:

In [15]:
region_2[region_2['product'] == 0.0]

,f0,f1,f2,product
68149,-0.865596,-1.615247,-4.126441,0.0


Видим, что второй регион имеет наименьшие показатели по целевому признаку. Это связано с тем, что в данных по скважинам в этом регионе имеет больше всего нулевых объемов скважин.
Причина возникновения нулей: вероятнее всего, это уже разработанные скважины, в которых, предположительно, уже нет запасов.
Поэтому, оставляем эти значения как есть.

**Вывод**

В процессе предобработки данных сделали следующее:
* удалили столбец `id` со всех датасетов
* оставили нулевые значения целевого признака.

Теперь приступим к обучению и проверке модели.

## Обучение и проверка модели<a id="3"></a>

In [16]:
def model(df: pd.DataFrame, num: int):
    # формируем фичи и прогнозируемый признак
    feature = df.drop('product', axis = 1)
    target = df['product']
    # делаем разбивку по выборкам
    # пропорции: обучающая - 75%, валидационная - 25%
    feature_train, feature_valid, target_train, target_valid = (
    train_test_split(feature, target, test_size = .25, random_state = 12345))
    # модель - линейная регрессия
    model = LinearRegression()
    model.fit(feature_train, target_train)
    predictions = model.predict(feature_valid)
    # расчет RMSE 
    rmse = mean_squared_error(target_valid, predictions) ** 0.5
    prediction = pd.Series(predictions)
    # вывод необходимых значений на экран
    print_res(rmse, prediction, num)
    # сохранение всех показателей
    return prediction, target_valid

In [17]:
def print_res(rmse: float, prediction: pd.Series, num: int):
    print(f'Средний запас предсказанного сырья в регионе №{num}: {prediction.mean():.3f}')
    print(f'Показатель RMSE: в регионе №{num}: {rmse:.3f}')

**Первый регион**:

In [18]:
prediction_0, target_0 = model(region_0, 1)

Средний запас предсказанного сырья в регионе №1: 92.593
Показатель RMSE: в регионе №1: 37.579


**Второй регион**:

In [19]:
prediction_1, target_1 = model(region_1, 2)

Средний запас предсказанного сырья в регионе №2: 68.729
Показатель RMSE: в регионе №2: 0.893


**Третий регион**:

In [20]:
prediction_2, target_2 = model(region_2, 3)

Средний запас предсказанного сырья в регионе №3: 94.965
Показатель RMSE: в регионе №3: 40.030


**Вывод**

В процессе обучения и получения прогнозов целевого показателя, а также получения необходимых метрик получили следующее:
* получили прогнозы и правильные ответы
* по офлайн-метрике RMSE установили, что модель лучше всего предсказала значения целевого показателя для второго региона. Предположительно, это связано с тем, что второй регион имеет показатели хуже, по сравнению с другими регионами и поэтому в датасете этого региона меньше выбивающихся значений, которые могут повлиять на значение метрики. 
* по регионам по среднему предсказанного сырья больше всего во втором регионе. Однако и метрику rmse имеет больше всех.

Теперь приступим к подготовке к расчету прибыли

## Подготовка к расчёту прибыли<a id="4"></a>

Расчитаем достаточный объем сырья для безубыточной разработки скважины. Прежде чем проводить расчеты определим константные значения.

In [21]:
# все необходимые константны пишем капсом
# чтобы различать переменные от констант

# выделяемый бюджет на регион: 10 млрд. руб.
BUDGET = 10_000_000_000
# лучшие скважины
BEST_BOREHOLES = 200
# доход с одной скважины (450 тыс. руб. за единицу продукта)
INCOME = 450_000
# количество исследуемых скважин
BOREHOLES = 500
# уровень риска (2.5%)
RISK_LOSS = .025 * 100

Теперь посчитаем объем сырья для безубыточной разработки скважины. Для этого необходимо определить следующее: 
стоимость на одну "лучшую" точку: разделить бюджет на разработку скважин для каждого региона на количество лучших точек.

In [22]:
DEVELOP_COST = BUDGET / BEST_BOREHOLES
print(f'Стоимость разработки одной скважины: {DEVELOP_COST}')

Стоимость разработки одной скважины: 50000000.0


Зная стоимость разработки одной скважины и какой доход может приносить одна скважина, посчитаем объем сырья, при котором скважина будет безубыточна: стоимость разработки одной скважины разделить на доход с одной скважины.

In [23]:
BOREHOLE_VOLUME = DEVELOP_COST / INCOME
print(f'Объем сырья, при котором скважина будет безубыточна: {BOREHOLE_VOLUME:.3f} т. баррелей')

Объем сырья, при котором скважина будет безубыточна: 111.111 т. баррелей


После того, как получили объем сырья, при котором скважина будет безубыточна, сравним полученный результат с данными из рассматриваемых регионов:

**Первый регион**:

In [24]:
print(f'Средний запас сырья в первом регионе: {target_0.mean()}')

Средний запас сырья в первом регионе: 92.07859674082927


In [25]:
print(f'Средний запас сырья во втором регионе: {target_1.mean()}')

Средний запас сырья во втором регионе: 68.72313602435997


In [26]:
print(f'Средний запас сырья в третьем регионе: {target_2.mean()}')

Средний запас сырья в третьем регионе: 94.88423280885438


На основе полученных расчетов установили, что все регионы, которые участвуют в исследовании - убыточны. Поэтому необходимо рассматривать только лучшие скважины во всех регионах.

Теперь посчитаем прибыль по выбранным скважинам и предсказаниям моделей.

<div class="alert alert-block alert-success">
<b>Успех:</b> Значение для безубыточной разработки посчитано верно, с выводом согласен
    
  
</div>

In [27]:
def revenue(predictions: pd.Series, target: pd.Series):
    # делаем сброс индексов для предсказаний и целевого признака
    predictions = predictions.reset_index(drop = True)
    target = target.reset_index(drop = True)
    # сортируем наши прогнозы по убыванию
    predict_sorted = predictions.sort_values(ascending = False)
    # выбираем 200 лучших скважин на основе прогноза:
    best_target = target[predict_sorted.index][:BEST_BOREHOLES]
    # получаем прибыль со скважины 
    #(доход за единицу продукта * сумму объема лучших скважин) - бюджет на разработку
    profit = ((INCOME*best_target.sum()) - BUDGET)/1e6
    return profit

In [28]:
def print_profit(prediction:pd.Series, target:pd.Series, num):
    print(f'Прибыль за сырье из скважин в регионе №{num}: {revenue(prediction, target):.3f}, млн. руб')

**Первый регион**

In [29]:
print_profit(prediction_0, target_0, 1)

Прибыль за сырье из скважин в регионе №1: 3320.826, млн. руб


**Второй регион**

In [30]:
print_profit(prediction_1, target_1, 2)

Прибыль за сырье из скважин в регионе №2: 2415.087, млн. руб


**Третий регион**

In [31]:
print_profit(prediction_2, target_2, 3)

Прибыль за сырье из скважин в регионе №3: 2710.350, млн. руб


**Вывод**

После расчета функции для определения прибыли по предсказаниям, определили, что наиболее прибыльным регионом оказался первый регион - его выручка составляет более 3 млрд. руб.

Самый худший показатель прибыльности у второго региона. Его прибыль составляет менее 2.5 млрд. руб.

После того, как посчитали прибыль для наших регионов. Перейдем к расчету прибыли, рисков, определению доверительного интервала. 

Для этого будем проводить эксперимент с реализацией техники Bootstrap. Для этого определим следующее:
* количество экспериментов: 1000
* количество элементов в подвыборке: 500
* с этой подвыборкой считаем ключевую метрику: потенциальную прибыль
* определяем доверительный интервал, в котором может с заданной вероятностью находиться истинное среднее значение прибыли с региона
* считаем долю риска (случаи, когда наша разработка скважин убыточна)
* считаем среднюю прибыль по региону

Теперь перейдем к нашему эксперименту.

## Расчёт прибыли и рисков <a id="5"></a>

In [32]:
def bootstrap(prediction: pd.Series, target: pd.Series, num = BOREHOLES):
    
    state = np.random.RandomState(12345)
    profit_val = []
    
    target = target.reset_index(drop = True)
    prediction = prediction.reset_index(drop = True)
    
    for i in range(1000):
        # формируем подвыборки:
        subsample_target = target.sample(n = num, replace = True, random_state = state)
        subsample_pred = prediction[subsample_target.index]
        # считаем ключевую метрику бизнеса
        profit_val.append(
        revenue(subsample_pred, subsample_target))
        
    profit = pd.Series(profit_val)
    mean_profit = profit.mean()
    (lower, upper) = (profit.quantile(.025), profit.quantile(.975))
    # уровень риска можно посчитать так:
    # risk_level = len(list(filter(lambda x: x < 0, profit))) / len(profit)
    # а можно посчитать сумму истинных предикатов
    risk_level = ((profit < 0).sum() / len(profit)) * 100
    return (lower, upper), mean_profit, risk_level

In [33]:
def print_bootstrap_res(prediction: pd.Series, 
                       target: pd.Series,
                        num: int):
    (lower, upper), mean_profit, risk_level = bootstrap(prediction, target)
    print(f'Основные результаты эксперимента для региона №{num}:')
    print(f'95%-ый доверительный интервал ключевой метрики: {(lower, upper)} млн. руб')
    print(f'Средняя прибыль: {mean_profit:.3f} млн. руб')
    print(f'Риск убытков: {risk_level:.2}%')
    if risk_level > RISK_LOSS:
        print('Вероятность риска высокая, регион не подходит')
    else:
        print('Вероятность риска низкая, обратите внимание на среднюю прибыль')

In [34]:
print_bootstrap_res(prediction_0, target_0, 1)

Основные результаты эксперимента для региона №1:
95%-ый доверительный интервал ключевой метрики: (-111.21554589049526, 909.7669415534225) млн. руб
Средняя прибыль: 396.165 млн. руб
Риск убытков: 6.9%
Вероятность риска высокая, регион не подходит


In [35]:
print_bootstrap_res(prediction_1, target_1, 2)

Основные результаты эксперимента для региона №2:
95%-ый доверительный интервал ключевой метрики: (33.820509398983624, 852.2894538660361) млн. руб
Средняя прибыль: 456.045 млн. руб
Риск убытков: 1.5%
Вероятность риска низкая, обратите внимание на среднюю прибыль


In [36]:
print_bootstrap_res(prediction_2, target_2, 3)

Основные результаты эксперимента для региона №3:
95%-ый доверительный интервал ключевой метрики: (-163.3504133956011, 950.3595749237995) млн. руб
Средняя прибыль: 404.404 млн. руб
Риск убытков: 7.6%
Вероятность риска высокая, регион не подходит


На основе полученных результатов видим, что второй регион - единственный из всех регионов, чей процент риска убытка от добычи в нем удовлетворяет требованиям ТЗ: вероятность риска меньше 2.5%. Более того, этот регион имеет наибольшую среднюю прибыль по сравнению с другими рассматриваемыми регионами.

## Общий вывод <a id="6"></a>

Была дана задача: построить модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой Bootstrap.

Для выполнения задания были данные по трем регионам.

В процессе знакомства с данными увидели следующие проблемы:

* имеются лишние для обучения данные (столбец `id`);
* в целевой метрике имеются нулевые значения объема сырья;

После предобработки данных получили следующие результаты:

* столбец `id` удален для всех регионов;
* нулевые значения оставили как есть, предположили, что у нас это разработанные скважины.

Далее разработали модель машинного обучения на основе линейной регресии, для которой задали следующие параметры:

* признаки: столбцы `f0`, `f1`, `f2`;
* целевой признак: столбец `product`.

* размер признаков - 75% от общего размера выборки;
* размер целевого признака - 25% от общего размера выборки.

После обучения модели и проверки офлайн-метрики RMSE получили следующие результаты:

* по офлайн-метрике RMSE установили, что модель лучше всего предсказала значения целевого показателя для второго региона. Предположительно, это связано с тем, что второй регион имеет показатели хуже, по сравнению с другими регионами и поэтому в датасете этого региона меньше выбивающихся значений, которые могут повлиять на значение метрики. 
* по регионам по среднему предсказанного сырья больше всего во втором регионе. Однако и метрику rmse имеет больше всех.

Далее определили минимальный объем сырья, при котором наша скважина будет безубыточна: 111.111 т. баррель. Затем сравнили полученный результат со средним объемом сырья для каждого региона и определили следующие выводы:

* Все регионы, которые участвуют в исследовании - убыточны; 
* Необходимо рассматривать только лучшие скважины во всех регионах.

Для определения потенциальной прибыли со скважин для каждого региона посчитали прибыль для 200 скважин, чей прогнозируемый объем максимален. Получили следующее:

* наиболее прибыльным регионом оказался первый регион - его выручка составляет более 3 млрд. руб.
* самый худший показатель прибыльности у второго региона. Его прибыль составляет менее 2.5 млрд. руб.

После того, как определили потенциальную прибыль, приступили к реализации эксперимента выбора наиболее прибыльного региона с помощью техники Bootstrap. Использованы для эксперимента следующие параметры:

* количество экспериментов: 1000
* количество элементов в подвыборке: 500
* с этой подвыборкой считаем ключевую метрику: потенциальную прибыль
* определяем доверительный интервал, в котором может с заданной вероятностью находиться истинное среднее значение прибыли с региона
* считаем долю риска (случаи, когда наша разработка скважин убыточна)
* считаем среднюю прибыль по региону

В результате получили следующее:

**Второй регион - единственный из всех регионов, чей процент риска убытка от добычи в нем удовлетворяет требованиям ТЗ: вероятность риска меньше 2.5%. Более того, этот регион имеет наибольшую среднюю прибыль по сравнению с другими рассматриваемыми регионами.**